In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('preprocessed_baseline.csv')

In [3]:
dataset

,OverallQual,GrLivArea,TotalBsmtSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,Foundation,Neighborhood,SalePrice
0,7,1710,856,2,8,2003,2003,225230.44,197965.77,208500
1,6,1262,1262,2,6,1976,1976,149805.71,238772.73,181500
2,7,1786,920,2,6,2001,2002,225230.44,197965.77,223500
3,7,1717,756,1,7,1915,1970,132291.08,210624.73,140000
4,8,2198,1145,2,9,2000,2000,225230.44,335295.32,250000
...,...,...,...,...,...,...,...,...,...,...
1455,6,1647,953,2,7,1999,2000,225230.44,192854.51,175000
1456,6,2073,1542,2,7,1978,1988,149805.71,189050.07,210000
1457,7,2340,1152,2,9,1941,2006,165959.17,210624.73,266500
1458,5,1078,1078,1,5,1950,1996,149805.71,145847.08,142125


In [4]:
y=dataset['SalePrice']
X=dataset.drop(columns=['SalePrice'])
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
#from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor

In [6]:
# Feature Scaling

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Initial tunning of Gradient Boosted Regressor

param_grid = {
    'loss': ['squared_error','absolute_error','huber','quantile'],
    'criterion':['squared_error','friedman_mse'],
    'learning_rate': [.07,.09,.11,.13,.15,.17],
    'max_depth': [10,20,30,40,50],
    'n_estimators': [50,75,100,125,150],
    'min_samples_split': [5,10,20,30,40],
    'min_impurity_decrease': [0, 0.005, 0.001, 0.005, 0.01],
}

gridSearch = GridSearchCV(GradientBoostingRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error')
gridSearch.fit(X_train, y_train)
print('Initial score: ', gridSearch.best_score_)
print('Initial parameteres: ', gridSearch.best_params_)

In [251]:
from sklearn import ensemble
params = {'n_estimators': 35, 'max_depth': 5, 'min_samples_split': 6,
          'learning_rate': .1, 'loss': 'squared_error'}
regressor = ensemble.GradientBoostingRegressor(**params)

regressor.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=5, min_samples_split=6, n_estimators=35)

In [252]:
# Predicting the Test set result
y_pred = regressor.predict(X_test)

In [253]:
from sklearn.metrics import mean_squared_error
from math import sqrt
sqrt(mean_squared_error(y_test, y_pred))

26952.048309222475

In [262]:
test = pd.read_csv('preprocessed_test.csv')

In [263]:
test

,OverallQual,GrLivArea,TotalBsmtSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,Neighborhood,Foundation
0,5,896,882.0,1,5,1961,1961,197965.77,149805.71
1,6,1329,1329.0,1,6,1958,1958,238772.73,149805.71
2,5,1629,928.0,2,6,1997,1998,197965.77,225230.44
3,6,1604,926.0,2,7,1998,1998,210624.73,225230.44
4,8,1280,1280.0,2,5,1992,1992,335295.32,225230.44
...,...,...,...,...,...,...,...,...,...
1454,4,1092,546.0,1,5,1970,1970,225379.84,149805.71
1455,4,1092,546.0,1,6,1970,1970,192854.51,149805.71
1456,5,1224,1224.0,1,7,1960,1996,189050.07,149805.71
1457,5,970,912.0,1,6,1992,1992,210624.73,225230.44


In [256]:
final_test = sc.transform(test)

C:\Users\Luke\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [257]:
# Predicting the Test set result
y_pred = regressor.predict(final_test)

In [258]:
y_pred

array([122412.32723863, 151775.71078783, 182239.70091603, ...,
       150672.35160863, 135456.03260216, 242359.41626744])

In [261]:
test['SalePrice'] = y_pred
test

,OverallQual,GrLivArea,TotalBsmtSF,FullBath,TotRmsAbvGrd,YearBuilt,YearRemodAdd,Neighborhood,Foundation,SalePrice
0,5,896,882.0,1,5,1961,1961,197965.77,149805.71,122412.327239
1,6,1329,1329.0,1,6,1958,1958,238772.73,149805.71,151775.710788
2,5,1629,928.0,2,6,1997,1998,197965.77,225230.44,182239.700916
3,6,1604,926.0,2,7,1998,1998,210624.73,225230.44,186332.763877
4,8,1280,1280.0,2,5,1992,1992,335295.32,225230.44,212810.745890
...,...,...,...,...,...,...,...,...,...,...
1454,4,1092,546.0,1,5,1970,1970,225379.84,149805.71,106856.981446
1455,4,1092,546.0,1,6,1970,1970,192854.51,149805.71,106856.981446
1456,5,1224,1224.0,1,7,1960,1996,189050.07,149805.71,150672.351609
1457,5,970,912.0,1,6,1992,1992,210624.73,225230.44,135456.032602


In [266]:
df = pd.read_csv('test.csv')
df['SalePrice'] = y_pred

In [267]:
df_out = df[['Id','SalePrice']]

In [268]:
df_out

,Id,SalePrice
0,1461,122412.327239
1,1462,151775.710788
2,1463,182239.700916
3,1464,186332.763877
4,1465,212810.745890
...,...,...
1454,2915,106856.981446
1455,2916,106856.981446
1456,2917,150672.351609
1457,2918,135456.032602


In [269]:
df_out.to_csv('gradientboosting_submission',index=False)